# Projet Gustave Malachane Kilian Violet

In [ ]:
!py -m pip install requests


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import urllib.request
import json
import pandas as pd
import requests
import shutil
import os

def download_image(url):

    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        with open("./images/"+os.path.basename(url), "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code


url = "https://query.wikidata.org/sparql?query=Select%20%3Fvolcan%20%3Fimage%20%3Flabel_pays%20%20%3Fhauteur%20%3Fhauteur_mer%20%3Fcoordonnees%20%3Flabel%20%7B%0A%20%20%3Fvolcan%20wdt%3AP31%20wd%3AQ169358%3B%0A%20%20%20%20%20%20%20%20wdt%3AP18%20%3Fimage%3B%0A%20%20%20%20%20%20%20%20wdt%3AP17%20%3Fpays%3B%0A%20%20%20%20%20%20%20%20wdt%3AP30%20%3Fcontinent%3B%0A%20%20%20%20%20%20%20%20wdt%3AP2660%20%3Fhauteur%3B%0A%20%20%20%20%20%20%20%20wdt%3AP2044%20%3Fhauteur_mer%3B%0A%20%20%20%20%20%20%20%20wdt%3AP625%20%3Fcoordonnees%3B%0A%20%20%20%20%20%20%20%20rdfs%3Alabel%20%3Flabel.%0A%20%20%0A%20%20%3Fpays%20rdfs%3Alabel%20%3Flabel_pays.%0A%20%20%0A%20%20FILTER(lang(%3Flabel)%3D%22en%22%20%26%26%20lang(%3Flabel_pays)%3D%22en%22)%0A%7D%0A%0ALIMIT%20150&format=json"
response = urllib.request.urlopen(url)
responsedata = json.loads(response.read().decode("utf-8"))

array = []
i = 1
for data in responsedata["results"]["bindings"]:
    
    response = download_image(data["image"]["value"])
    i += 1
    
    #array.append([data["label"]["value"], data["image"]["value"]])
dataframe = pd.DataFrame(array, columns=["label", "image"])
print(dataframe)

Empty DataFrame
Columns: [label, image]
Index: []


In [5]:
import os
import json
from PIL import Image
import exifread
import numpy as np
from sklearn.cluster import KMeans
from PIL import Image


def get_dominant_colors(image_path, k=5):
    """ Extrait les k couleurs dominantes d'une image avec K-Means """
    with Image.open(image_path) as img:
        img = img.resize((100, 100))  # Réduire la taille pour accélérer le clustering
        img_array = np.array(img).reshape(-1, 3)  # Convertir en tableau de pixels

    # Clustering des couleurs
    kmeans = KMeans(n_clusters=k, n_init=10)
    kmeans.fit(img_array)

    # Convertir les couleurs en hexadécimal
    colors = ['#%02x%02x%02x' % tuple(map(int, color)) for color in kmeans.cluster_centers_]
    return colors

# Dossier contenant les images
image_folder = "images"
metadata_list = []

# Parcourir chaque fichier image
for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)

    try:
        # Ouvrir l'image avec PIL
        with Image.open(image_path) as img:
            width, height = img.size
            format_img = img.format

            colors = get_dominant_colors(image_path)

            # Déterminer l'orientation
            if width > height:
                orientation = "Paysage"
            elif height > width:
                orientation = "Portrait"
            else:
                orientation = "Carré"

        # Lire les métadonnées EXIF
        with open(image_path, "rb") as img_file:
            tags = exifread.process_file(img_file, details=False)
            date_creation = tags.get("EXIF DateTimeOriginal", "Inconnu")
            camera_model = tags.get("Image Model", "Inconnu")

        # Stocker les métadonnées dans un dictionnaire
        metadata = {
            "nom": image_name,
            "chemin": image_path,
            "taille": f"{width}x{height}",
            "format": format_img,
            "orientation": orientation,
            "date_creation": str(date_creation),
            "modele_camera": str(camera_model),
            "colors": colors,
        }

        metadata_list.append(metadata)
        print(f"✅ Métadonnées extraites pour : {image_name}")

    except Exception as e:
        print(f"⚠️ Erreur avec {image_name} : {e}")

# Sauvegarder les métadonnées en JSON
with open("metadata.json", "w", encoding="utf-8") as json_file:
    json.dump(metadata_list, json_file, indent=4, ensure_ascii=False)

print("\n📄 Métadonnées enregistrées dans metadata.json")


c:\Users\gusti\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] Le fichier spécifié est introuvable
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\gusti\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "c:\Users\gusti\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\gusti\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 1036, in _

✅ Métadonnées extraites pour : 00%20126%202680%20Vulcano%20Calbuco%20-%20Chile.jpg
✅ Métadonnées extraites pour : 006Spirit%20Lake%20pre%20resize%20%2822027452382%29.jpg
✅ Métadonnées extraites pour : 2006-05-21-153901%20Iceland%20St%C3%B3rin%C3%BApur.jpg
✅ Métadonnées extraites pour : 981012-Damavand-South-IMG%209861-2.jpg
✅ Métadonnées extraites pour : Acotango.jpg
✅ Métadonnées extraites pour : Aerial%20image%20of%20Vulcano%20%28view%20from%20the%20east%29.jpg
✅ Métadonnées extraites pour : Ampato023.jpg
✅ Métadonnées extraites pour : AmpatoSabancaya.jpg
✅ Métadonnées extraites pour : Antisana.jpg
✅ Métadonnées extraites pour : Baitou%20Mountain%20Tianchi.jpg
✅ Métadonnées extraites pour : Cabaray.jpg
✅ Métadonnées extraites pour : Calbuco%2011%20Feb%202010%20zoomed%20in.jpg
✅ Métadonnées extraites pour : Caldera%20Mt%20Tambora%20Sumbawa%20Indonesia.jpg
✅ Métadonnées extraites pour : Carihuairazo1996.jpg
✅ Métadonnées extraites pour : Cerro%20paniri%20and%20volcan%20san%20pablo%20ch

In [ ]:
import json
import random

# Charger les volcans depuis un fichier JSON
with open("metadata.json", "r", encoding="utf-8") as f:
    volcans = json.load(f)

# Vérifier le nombre total de volcans
nombre_total_volcans = len(volcans)
print(f"Nombre total de volcans chargés : {nombre_total_volcans}")

# Génération de 50 utilisateurs fictifs
utilisateurs = [
    {"id": i, "nom": f"Utilisateur_{i}", "volcans_consultés": []}
    for i in range(1, 51)
]

# Attribution aléatoire des volcans consultés (en enregistrant uniquement le nom)
for utilisateur in utilisateurs:
    nb_volcans = random.randint(5, 20)  # Chaque utilisateur consulte entre 5 et 20 volcans
    volcans_choisis = random.sample(volcans, min(nb_volcans, nombre_total_volcans))
    
    # Stocker uniquement le nom des volcans
    utilisateur["volcans_consultés"] = [volcan["nom"] for volcan in volcans_choisis]

# Sauvegarde des données dans un fichier JSON
with open("utilisateurs_volcans.json", "w", encoding="utf-8") as f:
    json.dump(utilisateurs, f, indent=4, ensure_ascii=False)

# Affichage d'un aperçu des 5 premiers utilisateurs
print(json.dumps(utilisateurs[:5], indent=4, ensure_ascii=False))  # Afficher les 5 premiers utilisateurs
